# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
0.983033,"""51689762846807…","""../../../../da…","""46914510581565…","""12431689806671…","""10652901717770…",false
0.144221,"""16461133420582…","""../../../../da…","""11799983601655…","""12431689806671…","""15963704228679…",true
0.679745,"""16552322722747…","""../../../../da…","""19269595648710…","""29464956565927…","""10652901717770…",false
1.0,"""38351223768400…","""../../../../da…","""11799983601655…","""95499880837905…","""10652901717770…",false
0.718982,"""10130355283249…","""../../../../da…","""46914510581565…","""95499880837905…","""76236776619674…",true
0.0,"""12559652277712…","""../../../../da…","""19269595648710…","""51482922100770…",null,false
0.02333,"""10015021219472…","""../../../../da…","""19269595648710…","""95499880837905…","""76236776619674…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""47804597050427…","""../../../../da…","""43383010512446…","""FirstMortgage3…","""30235165729645…","""Rejected"""
0.144221,"""25511619055644…","""../../../../da…","""18111356444332…","""FirstMortgage3…","""58252810053289…","""Accepted"""
0.679745,"""72491287899039…","""../../../../da…","""69703925040601…","""MoneyMarketSav…","""30235165729645…","""Rejected"""
1.0,"""11115734777272…","""../../../../da…","""18111356444332…","""BasicChecking""","""30235165729645…","""Rejected"""
0.718982,"""10656945577576…","""../../../../da…","""43383010512446…","""BasicChecking""","""16398051003644…","""Accepted"""
0.0,"""68345507509995…","""../../../../da…","""69703925040601…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""17736169361314…","""../../../../da…","""69703925040601…","""BasicChecking""","""16398051003644…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""55275336157785…","""../../../../da…","""27438793365700…","""FirstMortgage3…","""11233431760178…","""Rejected"""
0.144221,"""18068626945197…","""../../../../da…","""12490735866104…","""FirstMortgage3…","""17569376679061…","""Accepted"""
0.679745,"""55679736956947…","""../../../../da…","""20505000020823…","""MoneyMarketSav…","""11233431760178…","""Rejected"""
1.0,"""16753445405249…","""../../../../da…","""12490735866104…","""BasicChecking""","""11233431760178…","""Rejected"""
0.718982,"""99878691141373…","""../../../../da…","""27438793365700…","""BasicChecking""","""75678176368550…","""Accepted"""
0.0,"""67100959078903…","""../../../../da…","""20505000020823…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""15822011144874…","""../../../../da…","""20505000020823…","""BasicChecking""","""75678176368550…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""77756542000237…","""36963874105424…","""FirstMortgage3…","""14759446273646…","""Rejected"""
0.144221,"""14904694988890…","""10425066542624…","""FirstMortgage3…","""44974785123927…","""Accepted"""
0.679745,"""17488650101433…","""98609686112332…","""MoneyMarketSav…","""14759446273646…","""Rejected"""
1.0,"""88606182586366…","""10425066542624…","""BasicChecking""","""14759446273646…","""Rejected"""
0.718982,"""11551178982123…","""36963874105424…","""BasicChecking""","""16419610527577…","""Accepted"""
0.0,"""60739378476500…","""98609686112332…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""60484834357487…","""98609686112332…","""BasicChecking""","""16419610527577…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""c73023dda51de1…","""../../../../da…","""af486bc1815c72…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
0.144221,"""5414b197fb87b3…","""../../../../da…","""3cdf7b1ae1342e…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
0.679745,"""892f5d61a9215e…","""../../../../da…","""806ef34eb8237f…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
1.0,"""c3aa807d9e927a…","""../../../../da…","""3cdf7b1ae1342e…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
0.718982,"""c2d492f9a05073…","""../../../../da…","""af486bc1815c72…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
0.0,"""b21059221605c4…","""../../../../da…","""806ef34eb8237f…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""80cbe489dcecd4…","""../../../../da…","""806ef34eb8237f…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
